In [ ]:
#1. 설치
#!git clone https://github.com/kh-kim/simple-ntc.git

In [ ]:
#!

#!pip install --user --upgrade pip
#!pip install --user pytorch-ignite
#!pip install --user fastai

In [ ]:
#2.기존 모델
!python train.py --model_fn ./model.pth --train /home/work/data/awkward_v2/data/albert_pretrain/sen_sen_label_v1_test.csv --cnn --gpu_id 0

In [ ]:
# 2-1. rnn
!python train.py --model_fn ./model.pth --train /home/work/data/awkward_v2/data/albert_pretrain/sen_sen_label_v1_test.csv --rnn --gpu_id 0

In [ ]:
#3. 새로 추가된 모델
!python train.py --model_fn ./model.pth --train /home/work/data/awkward_v2/data/albert_pretrain/sen_sen_label_v1_test.csv --awkward --gpu_id 0

In [ ]:
#3-1. 새로 추가된 모델
!python train.py --model_fn ./model.pth --train /home/work/data/awkward_v2/data/albert_pretrain/sen_sen_label_v1_test_v2.csv --awkward --gpu_id 0

In [ ]:
#3-1. 새로 추가된 모델
!python train.py --model_fn ./model.pth --train /home/work/data/awkward_v2/data/albert_pretrain/sen_sen_label_v1_test_v2.csv --awkward --gpu_id 0

In [ ]:
!nohup python train.py --model_fn ./model_gpu_1g_v1.pth --train /home/work/data/awkward_v2/data/albert_pretrain/xaa --awkward --gpu_id 0 & > awkward_train_0626_1g_v1.log

In [ ]:
python train.py --model_fn ./model_gpu_1g_1layer_v3.pth --train /home/work/data/awkward_v2/data/albert_pretrain/xaa.tsv --awkward --gpu_id 0

# 모델불러오기 & 추론

In [85]:
from classify import *
from engine.models.awkward import awkwardClassifier
from konlpy.tag import Okt

import torch
import argparse

In [86]:
path = "./"
model_name = "model_gpu_nsp_cosine_v7.pth"
use_eos=False
n_classes = 2

In [87]:
saved_data = torch.load(path + model_name, map_location = 'cuda:0' if torch.cuda.is_available() else 'cpu')

config = saved_data['config']
awkward_best = saved_data['awkward']
vocab1 = saved_data['vocab1']
vocab2 = saved_data['vocab2']
classes = saved_data['classes']

In [88]:
vocab1_size = len(vocab1)
vocab2_size = len(vocab2)

In [89]:
def define_field():
    '''
    To avoid use DataLoader class, just declare dummy fields. 
    With those fields, we can retore mapping table between words and indice.
    '''
    return (
        data.Field(sequential = True,
                              use_vocab = True,
                              batch_first = True,
                              include_lengths = False,
                              eos_token='<EOS>' if use_eos else None,
                              pad_first=True
                              ),
        data.Field(sequential = True,
                              use_vocab = True,
                              batch_first = True,
                              include_lengths = False,
                              eos_token='<EOS>' if use_eos else None,
                              pad_first=True
                              ),        
        data.Field(sequential = False,
                             preprocessing = lambda x: int(x),
                                 use_vocab = True,
                                init_token = None,
                                 unk_token = None
                              )
    )

In [90]:
text1_field, text2_field, label_field = define_field()

In [91]:
text1_field.vocab = vocab1
text2_field.vocab = vocab2 
label_field.vocab = classes

In [92]:
line1 = "코로나 극혐"
line2 = "다양한 기획들이 퍼즐처럼 맞추어 지듯이 객관적으로 필요한 절차에 도움이 될 자료들을 준비할 것입니다."

In [93]:
lines1 = []
lines2 = []

In [94]:
lines1 += [line1.strip().split(' ')]
lines2 += [line2.strip().split(' ')]

In [95]:
with torch.no_grad():
    x1 = text1_field.numericalize(
    text1_field.pad(lines1),
    device='cuda:0' if torch.cuda.is_available() else 'cpu',
    )
    
    x2 = text2_field.numericalize(
    text2_field.pad(lines2),
    device='cuda:0' if torch.cuda.is_available() else 'cpu',
    )

In [96]:
model = awkwardClassifier(
    ntoken = vocab1_size,
    ntoken2 = vocab2_size,
    ninp = config.word_vec_size,
    n_classes=n_classes,
    nhead = config.nhead, 
    nhid = config.nhid, 
    nlayers = config.nlayers,
    cos = config.cos,
    dropout=config.dropout,
)

In [97]:
ensemble = []
model.load_state_dict(awkward_best)
ensemble += [model]

In [ ]:
if config.gpu_id >= 0:
    model.cuda(config.gpu_id)
    
model.eval()

In [ ]:
y_hat = []

In [ ]:
def pad_to_maxseq_to_batch(batch, max_length, device=-1):
    
    if batch.size(1) >= max_length:
        batch = batch[:, :max_length]
        
    else:
        pad_num = max_length - batch.size(1)
        pad = torch.ones(batch.size(0), pad_num, device=device) # pad 값이 vocab index 1이라는 가정.
        batch = torch.cat([pad.long(), batch], dim=-1)

    return batch

In [ ]:
x1 = pad_to_maxseq_to_batch(x1, config.max_length, device='cuda:0' if torch.cuda.is_available() else 'cpu')
x2 = pad_to_maxseq_to_batch(x2, config.max_length, device='cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
print(model(x1, x2)[0])
result = torch.argmax(model(x1, x2)[0], dim=-1)
#model(x1, x2)

In [ ]:
# 1이면 정상
result

# 시행착오

In [ ]:
from classify import *
import argparse

In [ ]:
from engine.models.awkward import awkwardClassifier
from konlpy.tag import Okt

okt = Okt()

use_eos=False
n_classes = 2

In [ ]:
model_name = 'model.pth'
path = "/home/work/data/awkward_v2/my_model/datajuny_model_v2/datajuny_model_v2/"

saved_data = torch.load(path + model_name,
                       map_location = 'cuda:0' if torch.cuda.is_available() else 'cpu')

train_config = saved_data['config']
rnn_best = saved_data['rnn']
cnn_best = saved_data['cnn']
awkward_best = saved_data['awkward']
vocab1 = saved_data['vocab1']
vocab2 = saved_data['vocab2']
classes = saved_data['classes']

vocab1_size = len(vocab1)
vocab2_size = len(vocab2)

def define_field():
    '''
    To avoid use DataLoader class, just declare dummy fields. 
    With those fields, we can retore mapping table between words and indice.
    '''
    return (
        data.Field(sequential = True,
                              tokenize=okt.morphs,
                              use_vocab = True,
                              batch_first = True,
                              include_lengths = False,
                              eos_token='<EOS>' if use_eos else None,
                              pad_first=True
                              ),
        data.Field(sequential = True,
                              tokenize=okt.morphs,
                              use_vocab = True,
                              batch_first = True,
                              include_lengths = False,
                              eos_token='<EOS>' if use_eos else None,
                              pad_first=True
                              ),        
        data.Field(sequential = False,
                             preprocessing = lambda x: int(x),
                                 use_vocab = True,
                                init_token = None,
                                 unk_token = None
                              )
    )


def read_text():
    '''
    Read text from standard input for inference.
    '''
    lines = []

    for line in sys.stdin:
        if line.strip() != '':
            lines += [line.strip().split(' ')]

    return lines


text1_field, text2_field, label_field = define_field()

text1_field.vocab = vocab1 # 학습된 vocab을 넣는다.
text2_field.vocab = vocab2 
label_field.vocab = classes

lines = read_text()

with torch.no_grad():

    # convert string to list of index
    x = text1_field.numericalize(
        lines,   # 패딩 함수 제외
        device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    )
    
    ensemble = []
    
    if awkward_best is not None:

        model = awkwardClassifier(
                ntoken = vocab1_size,
                ntoken2 = vocab2_size,
                ninp = train_config.word_vec_size,
                n_classes=n_classes,
                nhead = train_config.nhead, 
                nhid = train_config.nhid, 
                nlayers = train_config.nlayers,
                dropout = train_config.dropout,
            )
        
        model.load_state_dict(awkward_best)
        ensemble += [model]

    
#device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    

In [ ]:
awkward_best

In [ ]:
sys.stdin

##  이하 테스트

In [ ]:
a = torch.randn(9,64,3)
b = torch.randn(9,64,3)

In [ ]:
#gen_outs = torch.exp(-torch.reduce_sum(torch.abs(gen_x - gen_x2), axis=1, keepdims=True))
#torch.abs(a - b)

torch.exp(-torch.sum(torch.abs(a - b), dim=2, keepdim=True), requires_grad=True)

In [ ]:
import numpy as np
np.array(test).shape

In [ ]:
import torch
a = torch.tensor([0, 1, 0, 0, 1, 1, 1, 0, 2])

In [ ]:
a.reshape(9, -1)

In [ ]:
a = np.array([[0], [1], [0], [0]])

In [ ]:
a.flatten()

In [ ]:
import torch
test =  torch.tensor([
        [[-2.1972, -2.1972],
         [-2.1972, -2.1972],
         [-2.1722, -2.4636],
         [-2.2590, -2.0194],
         [-2.2590, -2.0194],
         [-2.1484, -1.9949]],
    
        [[-2.1972, -2.1972],
         [-2.1972, -2.1972],
         [-2.1722, -2.4636],
         [-2.2590, -2.0194],
         [-2.2590, -2.0194],
         [-2.1484, -1.9949]],
    
        [[-2.1972, -2.1972],
         [-2.1972, -2.1972],
         [-2.4241, -1.1426],
         [-2.2590, -2.0194],
         [-2.2590, -2.0194],
         [2.1484, -1.9949]]])

In [ ]:
test.shape(dim=-1)

In [ ]:
torch.argmax(test, dim=-1)

In [ ]:
a = [[-1.0981e+00, -4.0572e-01, -1.9729e+01],
        [-1.3451e+00, -3.0182e-01, -1.9530e+01],
        [-2.6087e+00, -7.6487e-02, -1.2722e+01],
        [-3.8712e-01, -1.1364e+00, -1.1096e+01],
        [-2.5002e+00, -8.5733e-02, -9.2695e+00],
        [-1.0667e+00, -4.2310e-01, -7.0658e+00],
        [-2.2130e+00, -1.1596e-01, -9.0048e+00],
        [-1.2529e+00, -3.3737e-01, -7.2923e+00],
        [-5.9003e+00, -5.1762e+00, -8.4233e-03]]

In [ ]:
import numpy as np
np.array(a).shape

In [ ]:
aa = torch.tensor([[[-1.4528, -1.1640, -0.7900],
         [-1.0151, -1.0604, -1.2334],
         [-1.0151, -1.0604, -1.2334],
         [-1.0151, -1.0604, -1.2334],
         [-1.0151, -1.0604, -1.2334],
         [-1.0373, -1.1923, -1.0728]],

        [[-1.0151, -1.0604, -1.2334],
         [-1.9611, -1.0075, -0.7049],
         [-1.0151, -1.0604, -1.2334],
         [-0.5327, -1.7920, -1.4011],
         [-0.5327, -1.7920, -1.4011],
         [-0.5327, -1.7920, -1.4011]],

        [[-1.0151, -1.0604, -1.2334],
         [-1.9611, -1.0075, -0.7049],
         [-1.0151, -1.0604, -1.2334],
         [-0.5327, -1.7920, -1.4011],
         [-0.5327, -1.7920, -1.4011],
         [-0.5327, -1.7920, -1.4011]],

        [[-1.0151, -1.0604, -1.2334],
         [-1.0151, -1.0604, -1.2334],
         [-1.0151, -1.0604, -1.2334],
         [-0.5327, -1.7920, -1.4011],
         [-0.5327, -1.7920, -1.4011],
         [-0.5327, -1.7920, -1.4011]],

        [[-1.0151, -1.0604, -1.2334],
         [-1.0151, -1.0604, -1.2334],
         [-1.0151, -1.0604, -1.2334],
         [-0.5327, -1.7920, -1.4011],
         [-0.5327, -1.7920, -1.4011],
         [-0.5327, -1.7920, -1.4011]],

        [[-1.0151, -1.0604, -1.2334],
         [-1.0151, -1.0604, -1.2334],
         [-0.5327, -1.7920, -1.4011],
         [-0.5327, -1.7920, -1.4011],
         [-0.5327, -1.7920, -1.4011],
         [-0.5327, -1.7920, -1.4011]]])

In [ ]:
aa.squeeze(2).shape

In [ ]:
torch.gather(1, 2).squeeze(0)

In [ ]:
aa.reshape(36,-1)

In [ ]:
a = torch.tensor([[[-1.4330, -0.8303, -1.1224],
         [-1.4330, -0.8303, -1.1224],
         [-1.4330, -0.8303, -1.1224],
         [-1.4330, -0.8303, -1.1224],
         [-1.4330, -0.8303, -1.1224],
         [-1.7831, -0.3082, -2.3317]],
        [[-1.4330, -0.8303, -1.1224],
         [-1.4330, -0.8303, -1.1224],
         [-1.4330, -0.8303, -1.1224],
         [-1.4330, -0.8303, -1.1224],
         [-1.4330, -0.8303, -1.1224],
         [-1.7831, -0.3082, -2.3317]],
        [[-1.4330, -0.8303, -1.1224],
         [-1.5234, -0.9641, -0.9146],
         [-1.4330, -0.8303, -1.1224],
         [-1.3189, -2.1232, -0.4895],
         [-1.3189, -2.1232, -0.4895],
         [-1.3189, -2.1232, -0.4895]]])

In [ ]:
a.shape

In [ ]:
te = torch.tensor([[[-1.1486, -1.0664, -1.0828],
         [-1.1486, -1.0664, -1.0828],
         [-1.1486, -1.0664, -1.0828],
         [-1.2105, -2.0290, -0.5613],
         [-1.2105, -2.0290, -0.5613],
         [-1.2105, -2.0290, -0.5613]],

        [[-1.1486, -1.0664, -1.0828],
         [-1.1486, -1.0664, -1.0828],
         [-1.2105, -2.0290, -0.5613],
         [-1.2105, -2.0290, -0.5613],
         [-1.2105, -2.0290, -0.5613],
         [-1.2105, -2.0290, -0.5613]]])

In [ ]:
torch.argmax(te, dim=-1)

In [ ]:
ww = torch.tensor([[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2],
        [0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:
ww.shape